In [ ]:
from tifffile import imread
from tifffile import imwrite
from tifffile import imsave
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle

In [ ]:
# import UAV, elevation, slope, and apparent electrical conductivity maps
UAV_20201204p = imread('file_name')
UAV_20210114p = imread('file_name')
UAV_20210122p = imread('file_name')
UAV_20210201p = imread('file_name')
UAV_20201204w = imread('file_name')
UAV_20210114w = imread('file_name')
UAV_20210122w = imread('file_name')
DEMp = imread('file_name')
DEMw = imread('file_name')
Slope_p = imread('file_name')
Slope_w = imread('file_name')
EBKp = imread('file_name')
EBKw = imread('file_name') 
UAV_20201204p = np.delete(UAV_20201204p, 0, 0)
UAV_20210122p = np.delete(UAV_20210122p, 0, 0)

In [ ]:
# check data shape and type
print('20201204p shape:',UAV_20201204p.shape, 'dtype:',UAV_20201204p.dtype)
print('20210114p shape:',UAV_20210114p.shape, 'dtype:',UAV_20210114p.dtype)
print('20210122p shape:',UAV_20210122p.shape, 'dtype:',UAV_20210122p.dtype)
print('20210201p shape:',UAV_20210201p.shape, 'dtype:',UAV_20210201p.dtype)
print('20201204w shape:',UAV_20201204w.shape, 'dtype:',UAV_20201204w.dtype)
print('20210114w shape:',UAV_20210114w.shape, 'dtype:',UAV_20210114w.dtype)
print('20210122w shape:',UAV_20210122w.shape, 'dtype:',UAV_20210122w.dtype)
print('DEMp shape:',DEMp.shape, 'dtype:',DEMp.dtype)
print('DEMw shape:',DEMw.shape, 'dtype:',DEMw.dtype)
print('Slope_p shape:',Slope_p.shape, 'dtype:',Slope_p.dtype)
print('Slope_w shape:',Slope_w.shape, 'dtype:',Slope_w.dtype)
print('EBKp shape:',EBKp.shape, 'dtype:',EBKp.dtype)
print('EBKw shape:',EBKw.shape, 'dtype:',EBKw.dtype)

In [ ]:
# flatten the first two dimension, use * to iteratively represent all the elements after [2:] 
UAV_20201204p_fl = UAV_20201204p.reshape(-1, *UAV_20201204p.shape[2:]) 
UAV_20210114p_fl = UAV_20210114p.reshape(-1, *UAV_20210114p.shape[2:])
UAV_20210122p_fl = UAV_20210122p.reshape(-1, *UAV_20210122p.shape[2:])
UAV_20210201p_fl = UAV_20210201p.reshape(-1, *UAV_20210201p.shape[2:])
UAV_20201204w_fl = UAV_20201204w.reshape(-1, *UAV_20201204w.shape[2:])
UAV_20210114w_fl = UAV_20210114w.reshape(-1, *UAV_20210114w.shape[2:])
UAV_20210122w_fl = UAV_20210122w.reshape(-1, *UAV_20210122w.shape[2:])
DEMp_fl = DEMp.reshape(-1,*DEMp.shape[2:])
DEMw_fl = DEMw.reshape(-1,*DEMw.shape[2:])
Slope_p_fl = Slope_p.reshape(-1,*Slope_p.shape[2:])
Slope_w_fl = Slope_w.reshape(-1,*Slope_w.shape[2:])
EBKp_fl = EBKp.reshape(-1,*EBKp.shape[2:])
EBKw_fl = EBKw.reshape(-1,*EBKw.shape[2:])

In [ ]:
# import the trained model
UAV_mlp_2021 = pickle.load(open('file_name', 'rb'))

In [ ]:
# organize the predictors
raster = UAV_20220121w_fl # need change!
pixels = raster.shape[0]
doy = 60 #20201204 is 12, 20210114 is 53, 20210122 is 61, 20210201 is 71 
        #20211123 is 1, 20211129 is 7, 20211209 is 17, 20220111 is 50, 20220121 is 60
DOY = np.array([[doy]*1]*pixels)
Elevation = DEMw_fl # need change!
Slope = Slope_w_fl # need change!
EC = EBKw_fl # need change!

NDVI = (raster[:,4] - raster[:,2])/ (raster[:,4] + raster[:,2])
GNDVI = (raster[:,4] - raster[:,1])/ (raster[:,4] + raster[:,1])
SR = raster[:,4]/raster[:,2]
MSAVI = (2*raster[:,4]+1-((2*raster[:,4]+1)**2-8*(raster[:,4]-raster[:,2]))**1/2)/2
OSAVI = (raster[:,4]-raster[:,2])/(raster[:,4]+raster[:,2]+0.16)
RG = raster[:,2]/raster[:,1]
ExG = 2 * raster[:,1] - raster[:,2] - raster[:,0]
EVI = 2.5 * (raster[:,4] - raster[:,2])/ (raster[:,4] + 6 * raster[:,2] - 7.5 * raster[:,0] + 1)
TCARI = 3 * ((raster[:,3] - raster[:,2]) - 0.2 * (raster[:,3] - raster[:,1]) * (raster[:,3]/raster[:,2]))
MCARI = ((raster[:,3] - raster[:,2]) - 0.2 * (raster[:,3] - raster[:,1])) * (raster[:,3]/raster[:,2])
MTVI1 = 1.2 * (1.2 * (raster[:,4] - raster[:,1]) - 2.5 * (raster[:,2] - raster[:,1]))
NDRE = (raster[:,4] - raster[:,3])/ (raster[:,4] + raster[:,3])
NGRDI = (raster[:,1] - raster[:,2])/ (raster[:,1] + raster[:,2])
CL_rededge = (raster[:,4]/raster[:,3]) - 1
DVI = raster[:,4] - raster[:,2]
VARI = (raster[:,1] - raster[:,2])/ (raster[:,1] + raster[:,2] - raster[:,0]) 
raster_n = np.concatenate([DOY,Elevation[...,np.newaxis],Slope[...,np.newaxis],EC[...,np.newaxis],
                           NDVI[...,np.newaxis],GNDVI[...,np.newaxis],
                           SR[...,np.newaxis],MSAVI[...,np.newaxis],OSAVI[...,np.newaxis],
                           RG[...,np.newaxis],ExG[...,np.newaxis],EVI[...,np.newaxis],
                           TCARI[...,np.newaxis],MCARI[...,np.newaxis],MTVI1[...,np.newaxis],
                           NDRE[...,np.newaxis],NGRDI[...,np.newaxis],CL_rededge[...,np.newaxis],
                           DVI[...,np.newaxis],VARI[...,np.newaxis]], axis=1)
raster_n.shape

In [ ]:
# data cleaning
raster_r_inf = np.where(raster_n > 3.00000000e+38, 0.00000000e+00, raster_n) # replace values over than 3.00000000e+38 with 0, otherwise stay the same as original
raster_clean = np.nan_to_num(raster_r_inf, nan=0.00000000e+00) #replace nan values with 0
print('minimum value:',np.min(raster_clean))
print('maximum value:',np.max(raster_clean))

In [ ]:
# standardize the predictors
df_2021 = pd.read_csv('CSV_file')
# train_test set splitting 
X_2021, y_2021 = df_2021.iloc[:,1:].values, df_2021.iloc[:,0].values
X_train_2021, X_test_2021, y_train_2021, y_test_2021 = train_test_split(X_2021, y_2021, 
                                                        test_size=0.3, 
                                    random_state=37, stratify=df_2021.iloc[:,2]) 
# get standardization scaler
scaler_st_2021 = StandardScaler()
X_train_2021_st = scaler_st_2021.fit_transform(X_train_2021[:,1:])
# standardization
raster_st = scaler_st_2021.transform(raster_clean)

In [ ]:
# create grapevine water status map
UAV_WS = UAV_rf_2022.predict(raster_st) # need change!
UAV_WS_rs = UAV_WS.reshape(7621, 8820) # need change!
raster5_r_inf = np.where(raster[:,5] > 3.00000000e+38, 0.00000000e+00, raster[:,5])
raster5_rs = raster5_r_inf.reshape(7621, 8820)
print('GWS_min:',np.min(UAV_WS_rs))
print('GWS_max:',np.max(UAV_WS_rs))
print('GWS_mean:',np.mean(UAV_WS_rs))
print('height_min:',np.min(raster5_rs))
print('height_max:',np.max(raster5_rs))
print('height_mean:',np.mean(raster5_rs))

In [ ]:
# export images
imsave('file_name', UAV_WS_rs, resolution=(1./0.05, 1./0.05)) # need change!
# add planarconfig='contig' for saving multibands